# 1.
Решите задачу бинарной классификации деревом принятия решений. Сравните результаты с предыдущими, полученными методом логистической регрессии, на основе изученных метрик. Задавая алгоритм для новой модели, укажите параметр random_state=0.

Подсказка:
Для прогноза вероятностей и итоговых классов вызовите методы predict() и predict_proba() модели.
А ещё не забудьте, что для передачи на вход функции расчёта метрики roc_auc нужен только вектор отнесения к классу 1 (а не вектор пар вероятностей для двух классов) — тут поможет конструкция [:,1].

In [3]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

# прочитаем из csv-файла данные о параметрах сетей и их устойчивости
electrical_grid = pd.read_csv(
    '../datasets/Electrical_Grid_Stability.csv', sep=';'
)
print('Размер датасета:', electrical_grid.shape)
electrical_grid.head()

# посмотрим, как соотносятся классы для нашего набора данных
electrical_grid['stability'].value_counts()

# разделим наши данные на признаки (матрица X) и целевую переменную (y)
X = electrical_grid.drop('stability', axis=1)
y = electrical_grid['stability']

# разделяем модель на обучающую и валидационную выборку
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# обучите StandartScaler на обучающей выборке
scaler = StandardScaler()
scaler.fit(X_train)

# Преобразуйте обучающий и валидационные наборы данных
X_train_st = scaler.transform(X_train)
X_test_st = scaler.transform(X_test)

# зададим алгоритм для нашей модели
model = LogisticRegression(solver='liblinear', random_state=0)

# обучим модель
model.fit(X_train_st, y_train)

# воспользуемся уже обученной моделью, чтобы сделать прогнозы
predictions = model.predict(X_test_st)
probabilities = model.predict_proba(X_test_st)[:, 1]

# выведем все изученные метрики
print('Метрики для логистической регрессии')
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, predictions)))
print('Precision: {:.2f}'.format(precision_score(y_test, predictions)))
print('Recall: {:.2f}'.format(recall_score(y_test, predictions)))
print('F1: {:.2f}'.format(f1_score(y_test, predictions)))
print('ROC_AUC: {:.2f}\n'.format(roc_auc_score(y_test, probabilities)))

# зададим алгоритм для новой модели на основе алгоритма решающего дерева
tree_model = DecisionTreeClassifier(random_state=0)

# обучите модель
tree_model.fit(X=X_train, y=y_train)


# воспользуемся уже обученной моделью, чтобы сделать прогнозы
tree_predictions = tree_model.predict(X=X_test)
tree_probabilities = tree_model.predict_proba(X=X_test)[:, 1]

# выведем все изученные метрики
print('Метрики для дерева принятия решения')
print('Accuracy: {:.2f}'.format(accuracy_score(y_true=y_test, y_pred=tree_predictions)))
print('Precision: {:.2f}'.format(precision_score(y_true=y_test, y_pred=tree_predictions)))
print('Recall: {:.2f}'.format(recall_score(y_true=y_test, y_pred=tree_predictions)))
print('F1: {:.2f}'.format(f1_score(y_true=y_test, y_pred=tree_predictions)))
print('ROC_AUC: {:.2f}'.format(roc_auc_score(y_true=y_test, y_score=tree_probabilities)))

Размер датасета: (10000, 13)
Метрики для логистической регрессии
Accuracy: 0.81
Precision: 0.77
Recall: 0.70
F1: 0.73
ROC_AUC: 0.89

Метрики для дерева принятия решения
Accuracy: 0.83
Precision: 0.77
Recall: 0.77
F1: 0.77
ROC_AUC: 0.82
